In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
円形電流ループの軸上磁束密度計算スクリプト

ループ半径 R [m]、ループ位置 z0 [m]、電流 I [A] を指定し、
軸上位置 z [m] における磁束密度 B_z [T] を計算します。

式:
    B_z(z) = μ0 * I * R^2
             -------------------
             2 * [R^2 + (z - z0)^2]^(3/2)
"""

import numpy as np
import plotly.graph_objects as go
import re

def awg_to_diameter(awg: int, unit: str = "mm") -> float:
    """
    AWG（American Wire Gauge）の文字列を受け取り、その導線の外径を返す。

    Parameters
    ----------
    awg_str : str
        AWGを表す文字列（例: "28AWG", "12 awg" など）。
    unit : str, optional
        返り値の単位。"mm"（デフォルト）または "inch" を指定。

    Returns
    -------
    float
        指定した単位での導線外径。

    Raises
    ------
    ValueError
        awg_str が "nAWG" の形式でない場合や、unit が "mm" でも "inch" でもない場合。
    """

    # AWGからインチ単位の直径を計算
    # d_inch = 0.005 in × 92^((36 − AWG) / 39)
    d_inch = 0.005 * 92 ** ((36 - awg) / 39)

    if unit.lower() == "inch":
        return d_inch
    elif unit.lower() == "mm":
        return d_inch * 25.4
    else:
        raise ValueError(f"不明な単位 '{unit}'。'mm' または 'inch' を指定してください。")


def magnetic_field_on_axis(z, R, I, z0=0.0):
    """
    軸上磁束密度 B_z を返す関数。

    Parameters
    ----------
    z : float or array_like
        軸上の評価位置 [m]
    R : float
        ループ半径 [m]
    I : float
        電流 [A]
    z0 : float, optional
        ループの中心位置 [m] （デフォルト: 0）

    Returns
    -------
    Bz : float or ndarray
        磁束密度 [T]
    """
    mu0 = 4 * np.pi * 1e-7  # 真空の透磁率 [H/m]
    dz = z - z0
    return mu0 * I * R**2 / (2 * (R**2 + dz**2)**1.5)

def define_coil_1layer(z_start, N_turn, d_winding, R_inner):
     
    z = z_start + d_winding/2 + np.arange(N_turn) * d_winding
    r = R_inner * np.ones_like(z) + d_winding/2
    Loop_R_Z = []
    for i in range(len(z)):
        Loop_R_Z.append([r[i], z[i]])
    return Loop_R_Z

def main_BCOIL_ORIGINAL(I = 20.0, shapefigshow=True):
    # --- ユーザー設定パラメータ ---
    # I  = 20.0    # 電流 [A]
    print(f"STEP0 : 電流 I = {I} A で計算を開始します。")
    # 各コイルの基本情報定義    
    BCOIL_Loop_R_Z = []
    BCOIL_windingAWG = [
        [16, 16, 16, 16, 16], # Bias Coil 1
        [16, 16, 18, 18, 18], # Bias Coil 2
        [18, 18, 18]          # Bias Coil 3
    ]        
    BCOIL_N_turn = [
        [130, 130, 130, 130, 130], # Bias Coil 1
        [130, 130, 160, 160, 160], # Bias Coil 2
        [160, 160, 160]  # Bias Coil 3
    ]
    BCOIL_Ri =[
        41.275,  # Bias Coil 1 [mm]
        65.845,  # Bias Coil 2 [mm]
        90.025   # Bias Coil 3 [mm] 
    ] 
    BCOIL_INFO = {
        "Rmin": [],
        "Rmax": [],
        "Rdist": [],
        "Zmin": [],
        "Zmax": [],
        "Zdist": []
        }
    # 全層数
    BCOIL_NCOIL = len(BCOIL_Ri) # コイル数
    BCOIL_NLayer = [len(layer) for layer in BCOIL_N_turn] # 各コイルの層数
    # 各コイルの線径 [mm]
    BCOIL_dwinding = []
    for i_coil in range(BCOIL_NCOIL):
        BCOIL_dwinding.append([awg_to_diameter(awg) for awg in BCOIL_windingAWG[i_coil]])

    
    print(f"STEP1 : コイル情報を定義します。")
    # 各コイルの座標定義
    for i_coil in range(BCOIL_NCOIL):
        # 各コイルのループ定義
        R_inner_calc = BCOIL_Ri[i_coil] * 1e-3  # コイルの内半径 [m]
        BCOIL_Loop_R_Z_1COIL = []
        
        for i_layer in range(len(BCOIL_N_turn[i_coil])):
            # コイルの各層ループの定義用変数
            z_start = np.mod(i_layer,2) * BCOIL_dwinding[i_coil][i_layer] / 2  * 1e-3  # 層の開始位置 [m]
            # ループの定義
            BCOIL_Loop_R_Z_1COIL.append(
                define_coil_1layer(z_start, 
                                   BCOIL_N_turn[i_coil][i_layer],
                                   BCOIL_dwinding[i_coil][i_layer] * 1e-3,
                                   R_inner_calc)
            )
            # 次の層の内径を計算
            if i_layer < len(BCOIL_N_turn[i_coil])-1:
                if BCOIL_dwinding[i_coil][i_layer] == BCOIL_dwinding[i_coil][i_layer + 1]:
                    # 次の層も線径が等しい場合
                    R_inner_calc += BCOIL_dwinding[i_coil][i_layer]*np.cos(np.radians(30)) * 1e-3
                else:
                    # 次の層と線径が異なる場合
                    R_inner_calc += BCOIL_dwinding[i_coil][i_layer] * 1e-3

        # コイル1つ分の座標リストをコイル情報リストに追加
        BCOIL_Loop_R_Z.append(BCOIL_Loop_R_Z_1COIL)
        BCOIL_INFO["Zmax"].append(np.max(BCOIL_Loop_R_Z_1COIL[:][:][1]))
        BCOIL_INFO["Zmin"].append(np.min(BCOIL_Loop_R_Z_1COIL[:][:][1]))
        BCOIL_INFO["Zdist"].append(np.max(BCOIL_Loop_R_Z_1COIL[:][:][1]) - np.min(BCOIL_Loop_R_Z_1COIL[:][:][1]))
        BCOIL_INFO["Rmax"].append(np.max(BCOIL_Loop_R_Z_1COIL[:][:][0]))
        BCOIL_INFO["Rmin"].append(np.min(BCOIL_Loop_R_Z_1COIL[:][:][0]))
        BCOIL_INFO["Rdist"].append(np.max(BCOIL_Loop_R_Z_1COIL[:][:][0]) - np.min(BCOIL_Loop_R_Z_1COIL[:][:][0]))

    # ユーザーから評価位置を入力（スカラー or カンマ区切りリスト）
    z_vals = np.linspace(np.min(BCOIL_INFO["Zmin"]) - np.max(BCOIL_INFO["Zdist"]), 
                         np.max(BCOIL_INFO["Zmax"]) + np.max(BCOIL_INFO["Zdist"]), 
                         200)
    Bz_eachlayer = np.zeros((len(z_vals), np.sum(BCOIL_NLayer)))


    print(f"STEP2: 各コイルの磁束密度 Bz を計算します。")
    # Bz の計算
    # 各コイルごとに
    for i_coil in range(BCOIL_NCOIL):
        # 各コイルの層ごとに
        for i_layer in range(BCOIL_NLayer[i_coil]):
            # Bz配列の計算結果格納先インデックスを設定
            i_bz = int(np.sum(BCOIL_NLayer[:i_coil]) + i_layer)
            # 各ループ位置毎に
            for i_loop in range(len(BCOIL_Loop_R_Z[i_coil][i_layer])):
                loopR_calc = BCOIL_Loop_R_Z[i_coil][i_layer][i_loop][0]
                loopZ_calc = BCOIL_Loop_R_Z[i_coil][i_layer][i_loop][1]
                # 各z評価位置毎に
                for i_z in range(len(z_vals)):
                    z_calc = z_vals[i_z]
                    # 磁束密度 Bz の計算
                    Bz_eachlayer[i_z][i_bz] += magnetic_field_on_axis(z_calc, loopR_calc, I, loopZ_calc)
    # 全コイルのBz和
    Bz = np.zeros((len(z_vals), BCOIL_NCOIL + 1))  # +1は全コイルの和用
    for i_coil in range(BCOIL_NCOIL):
        # 各コイルのBzを和に追加
        Bz[:, i_coil] = Bz_eachlayer[:, int(np.sum(BCOIL_NLayer[:i_coil])):int(np.sum(BCOIL_NLayer[:i_coil+1]))].sum(axis=1)
    # 全コイルのBzを和に追加
    Bz[:, BCOIL_NCOIL] = Bz_eachlayer.sum(axis=1)

    # -----------------------------
    
    # -----------------------------
    print(f"STEP3: 計算結果をプロットします。")
    # 計算結果をプロット
    fig = go.Figure()
    # 各コイルのBzをプロット
    for i_coil in range(BCOIL_NCOIL):
        fig.add_trace(go.Scatter(
            x=z_vals * 1e3,  # mmに変換
            y=Bz[:, i_coil] * 1e3,  # mTに変換
            mode='lines',
            name=f'Coil {i_coil + 1}',
            line=dict(width=2)
        ))
    # 全コイルの和をプロット
    fig.add_trace(go.Scatter(
        x=z_vals * 1e3,  # mmに変換
        y=Bz[:, BCOIL_NCOIL] * 1e3,  # mTに変換
        mode='lines',
        name='Total',
        line=dict(width=2, dash='dash')
    ))
    # レイアウト調整
    fig.update_layout(
        title='Signals (second half)',
        xaxis_title='z [mm]',
        yaxis_title='Bz [mT]'
    )
    fig.show()
    
    if shapefigshow is True:
        # -----------------------------
        # -----------------------------
        print("STEP4: 各ループをShapeの円で描画します。")
        # 1) shapes リストに全定義をため込む
        shapes = []
        for i_coil in range(BCOIL_NCOIL):
            for i_layer in range(BCOIL_NLayer[i_coil]):
                loops = BCOIL_Loop_R_Z[i_coil][i_layer]
                dw = BCOIL_dwinding[i_coil][i_layer]  # mm
                color_line = f'rgba({i_coil*50}, {i_layer*50}, 150, 0.5)'
                color_fill = f'rgba({i_coil*50}, {i_layer*50}, 150, 0.3)'

                for idx, (r_m, z_m) in enumerate(loops, start=1):
                    # 進捗表示は必要なら間引き (例: 1000ループごと)
                    if idx % 1000 == 0:
                        print(f"Coil {i_coil+1}, Layer {i_layer+1}, Loop {idx}/{len(loops)}", end='\r')

                    rm = r_m * 1e3  # m→mm
                    zm = z_m * 1e3
                    shapes.append(dict(
                        type="circle", xref="x", yref="y",
                        x0=rm - dw/2, y0=zm - dw/2,
                        x1=rm + dw/2, y1=zm + dw/2,
                        line=dict(color=color_line, width=1),
                        fillcolor=color_fill
                    ))

        # 2) 一度に Figure を生成し、layout に渡す
        fig2 = go.Figure()
        fig2.update_layout(
            title="各ループをShapeの円で描画",
            shapes=shapes,
            xaxis=dict(
                title="R [mm]",
                scaleanchor="y",
                scaleratio=1,
                range=[
                    (np.min(BCOIL_INFO["Rmin"]) - np.max(BCOIL_INFO["Rdist"])/2) * 1e3,
                    (np.max(BCOIL_INFO["Rmax"]) + np.max(BCOIL_INFO["Rdist"])/2) * 1e3
                ]
            ),
            yaxis=dict(
                title="Z [mm]",
                range=[
                    (np.min(BCOIL_INFO["Zmin"]) - np.max(BCOIL_INFO["Zdist"])/2) * 1e3,
                    (np.max(BCOIL_INFO["Zmax"]) + np.max(BCOIL_INFO["Zdist"])/2) * 1e3
                ]
            ),
            showlegend=False,
            width=1000,
            height=1000
        )

        fig2.show()

    
    print(f"STEP5: 計算が完了しました。")
    return (Bz, z_vals)
    

In [58]:
Bz, z_vals = main_BCOIL_ORIGINAL(I=5.0, shapefigshow=False)

STEP0 : 電流 I = 5.0 A で計算を開始します。
STEP1 : コイル情報を定義します。
STEP2: 各コイルの磁束密度 Bz を計算します。
STEP3: 計算結果をプロットします。


STEP5: 計算が完了しました。


In [59]:
Bz, z_vals = main_BCOIL_ORIGINAL(I=5.0, shapefigshow=True)

STEP0 : 電流 I = 5.0 A で計算を開始します。
STEP1 : コイル情報を定義します。
STEP2: 各コイルの磁束密度 Bz を計算します。
STEP3: 計算結果をプロットします。


STEP4: 各ループをShapeの円で描画します。


STEP5: 計算が完了しました。


In [71]:
def make_sequence(n: int) -> list[float]:
    """
    正の整数 n を受け取り、
    最小値 -n/2、最大値 +n/2 の等差数列（要素数 n）を返す。

    例:
      n = 5 → [-2.5, -1.25, 0.0, 1.25, 2.5]
      n = 1 → [0.0]
    """
    if n <= 0:
        raise ValueError("n は正の整数でなければなりません")
    # 要素が1つだけなら 0.0 を返す
    if n == 1:
        return [0.0]

    min_val = np.ceil(n / 2) - n
    max_val =  n / 2
    # 両端を含む等差数列のステップ幅
    step = 1

    return [min_val + i * step for i in range(n)]


# ── 動作例 ──
if __name__ == "__main__":
    for n in [1, 3, 5, 10]:
        seq = make_sequence(n)
        print(f"n={n} → {seq}")

n=1 → [0.0]
n=3 → [-1.0, 0.0, 1.0]
n=5 → [-2.0, -1.0, 0.0, 1.0, 2.0]
n=10 → [-5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0]


In [ ]:
for i in range(5, 10):
    a = np.arange(i) - np.ceil(i/2)
  print("i = {}, i/2 = {}".format(i, np.ceil(i/2)))

i = 0, i/2 = 0.0
i = 1, i/2 = 1.0
i = 2, i/2 = 1.0
i = 3, i/2 = 2.0
i = 4, i/2 = 2.0
i = 5, i/2 = 3.0
i = 6, i/2 = 3.0
i = 7, i/2 = 4.0
i = 8, i/2 = 4.0
i = 9, i/2 = 5.0
